# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    regex = re.compile(r'\w+$|\w+\*$')
    y = re.compile(r'^\w+')
    w = re.compile(r'\[\w+\s\w+\]|\W')
    nh = cities.iloc[:-1,[0,1,5]]
    nh= nh.sort_values("NHL")
    nh = nh.rename(columns={'NHL':'team'})
    nh["team"] = nh["team"].replace({w:''}, regex=True)
    nh_1 = nhl_df[(nhl_df['year']) == 2018]
    nh.drop(nh[(nh['team'] == "")].index, inplace=True)
    nh_1["team"] = nh_1["team"].replace({'\*':''}, regex=True)
    nh_1["team"] = nh_1["team"].apply(lambda x: regex.findall(x)[0])
    nh_1["team"] = nh_1["team"].replace({'Jackets':'BlueJackets','Leafs':'MapleLeafs','Knights':'GoldenKnights','Ducks':'KingsDucks','Wings':'RedWings'}, regex=True)
    nh["team"] = nh["team"].apply(lambda x: y.findall(x)[0])
    WR = pd.pivot_table(nh_1,index=["team"],values=['W', 'L' ],aggfunc=np.sum)
    WR = WR.sort_values('team')
    WR = WR.drop(['Division'], axis=0)
    WR = WR.astype({'W': 'int32', 'L': 'int32'})
    WR["WinRate"] = WR['W'] / (WR['W']+WR['L'])
    nh_1.set_index("team", drop=True, append=False, inplace=True, verify_integrity=False)
    nh.set_index("team",drop=True, append=False, inplace=True, verify_integrity=False)
    f = pd.merge(WR, nh, on='team', how='outer')
    f= f.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population'})
    Vn = [f.loc[['Devils','Islanders','Rangers']]['L'].sum(),f.loc[['Devils','Islanders','Rangers']]['W'].sum(),
          f.loc[['Devils','Islanders','Rangers']]['WinRate'].sum(),f.loc[['RangersIslandersDevils']]['local'],
          f.loc[['RangersIslandersDevils']]['population']]
    f.loc[['RangersIslandersDevils']]=Vn
    Vla =[f.loc[['Kings','KingsDucks']]['L'].sum(),f.loc[['Kings','KingsDucks']]['W'].sum(),
          f.loc[['Kings','KingsDucks']]['WinRate'].sum(),f.loc[['KingsDucks']]['local'],
          f.loc[['KingsDucks']]['population']]
    f.loc[['KingsDucks']]=Vla
    f = f.dropna()
    f = f.astype({'population':'int32'})
    a = f['population']
    b = f['WinRate']
    
    population_by_region = a # pass in metropolitan area population from cities
    win_loss_by_region = b # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()
nhl_correlation()

(0.7752196800466112, 1.2711644398637227e-06)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    NBA = cities.iloc[:-1,[0,1,4]]
    NBA = NBA.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population' ,'NBA':'team'})
    regex = re.compile(r'\s\w+\s\(')
    y = re.compile(r'^\w+')
    w = re.compile(r'\[\w+\s\w+\]|\W')
    NBA["team"] = NBA["team"].replace({w:''}, regex=True)
    NBA.drop(NBA[(NBA['team'] == "")].index, inplace=True)
    nba = nba_df[(nba_df['year']) == 2018]
    nba = nba.iloc[:,[0,3]]
    nba["team"] = nba["team"].replace({'\*':''}, regex=True)
    nba["team"] = nba["team"].apply(lambda x: regex.findall(x)[0])
    nba["team"] = nba["team"].replace({'\(':''}, regex=True)
    nba["team"] = nba["team"].replace({'\s':''}, regex=True)
    nba['team'] = nba['team'].replace({'Blazers':'TrailBlazers','Clippers':'LakersClippers'},regex=True)
    nba.set_index("team",drop=True, append=False, inplace=True, verify_integrity=False)
    NBA.set_index("team",drop=True, append=False, inplace=True, verify_integrity=False)
    nba = nba.sort_values('team')
    NBA = NBA.sort_values('team')
    nba = nba.astype({'W/L%': 'float32'})
    NBA = NBA.astype({'population': 'int32'})
    e = pd.merge(NBA, nba, on='team', how='outer')
    LA = [e.loc['LakersClippers']['local']
         ,e.loc['LakersClippers']['population'],
         (e.loc[['LakersClippers','Lakers']]['W/L%'].sum())/2]
    NY = [e.loc['KnicksNets']['local'],
          e.loc['KnicksNets']['population'],
          (e.loc[['Knicks','Nets']]['W/L%'].sum())/2]
    e.loc['KnicksNets'] = NY
    e.loc['LakersClippers'] = LA
    e = e.dropna()
  
    a = e['population']
    b = e['W/L%']
    population_by_region = a # pass in metropolitan area population from cities
    win_loss_by_region = b # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()
nba_correlation()

(-0.1763634967664775, 0.36932108848389705)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    MLB = cities.iloc[:-1,[0,1,3]]
    mlb = mlb_df[(mlb_df['year']) == 2018]
    mlb = mlb.iloc[:,[0,3]]
    w = re.compile(r'\[\w+\s\w+\]|\W')
    regex = re.compile(r'\w+$')
    MLB = MLB.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population' ,'MLB':'team'})
    MLB["team"] = MLB["team"].replace({w:''}, regex=True)
    MLB.drop(MLB[(MLB['team'] == "")].index, inplace=True)
    mlb["team"] = mlb["team"].replace({'Boston Red Sox':'RedSox','Tampa Bay Rays':'Rays',
                                       'Toronto Blue Jays':'BlueJays','Chicago White Sox':'WhiteSox',}, regex=True)
    mlb["team"] = mlb["team"].apply(lambda x: regex.findall(x)[0])
    mlb.set_index("team", drop=True, append=False, inplace=True, verify_integrity=False)
    MLB.set_index("team",drop=True, append=False, inplace=True, verify_integrity=False)
    MLB = MLB.astype({'population':'int32'})
    mlb = mlb.astype({'W-L%': 'float32'})
    c = pd.merge(MLB, mlb, on='team', how='outer')
    NY = [c.loc[['YankeesMets']]['local'],c.loc[['YankeesMets']]['population'],(c.loc[['Yankees','Mets']]['W-L%'].sum())/2]
    c.loc[['YankeesMets']]=NY
    LA = [c.loc[['DodgersAngels']]['local'],c.loc[['DodgersAngels']]['population'],(c.loc[['Dodgers','Angels']]['W-L%'].sum())/2]
    c.loc[['DodgersAngels']]=LA
    SF = [c.loc[['GiantsAthletics']]['local'],c.loc[['GiantsAthletics']]['population'],(c.loc[['Giants','Athletics']]['W-L%'].sum())/2]
    c.loc[['GiantsAthletics']]=SF
    C=[c.loc[['CubsWhiteSox']]['local'],c.loc[['CubsWhiteSox']]['population'],(c.loc[['Cubs','WhiteSox']]['W-L%'].sum())/2]
    c.loc[['CubsWhiteSox']]=C
    c = c.dropna()

    a = c['population']
    b = c['W-L%']
    population_by_region = a # pass in metropolitan area population from cities
    win_loss_by_region = b # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()
mlb_correlation()

(0.15003735842225893, 0.46442832119114447)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    NFL = cities.iloc[:-1,[0,1,2]]
    nfl = nfl_df[(nfl_df['year']) == 2018]
    nfl = nfl.iloc[:,[12,13]]
    NFL = NFL.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population' ,'NFL':'team'})
    w = re.compile(r'\[\w+\s\w+\]|\W')
    regex = re.compile(r'\w+$|\w+\*|\w+\+')
    y = re.compile(r'\w+\s\w+')
    NFL["team"] = NFL["team"].replace({w:''}, regex=True)
    NFL.drop(NFL[(NFL['team'] == "")].index, inplace=True)
    nfl["team"] = nfl["team"].apply(lambda x: regex.findall(x)[0])
    nfl["team"] = nfl["team"].replace({w:''}, regex=True)
    nfl["W-L%"] = nfl["W-L%"].replace({y:''}, regex=True)
    a= nfl.drop(nfl[(nfl['W-L%'] == "")].index, inplace=True)
    nfl["W-L%"] = nfl["W-L%"].replace({'\.':'0.'}, regex=True)
    nfl = nfl.astype({'W-L%': 'float32'})
    NFL = NFL.astype({'population':'int32'})
    nfl.set_index("team", drop=True, append=False, inplace=True, verify_integrity=False)
    NFL.set_index("team",drop=True, append=False, inplace=True, verify_integrity=False)
    d = pd.merge(NFL, nfl, on='team', how='outer')
    NY = [d.loc[['GiantsJets']]['local'],d.loc[['GiantsJets']]['population'],(d.loc[['Giants','Jets']]['W-L%'].sum())/2]
    d.loc[['GiantsJets']]=NY
    LA = [d.loc[['RamsChargers']]['local'],d.loc[['RamsChargers']]['population'],(d.loc[['Rams','Chargers']]['W-L%'].sum())/2]
    d.loc[['RamsChargers']] = LA
    SF = [d.loc[['49ersRaiders']]['local'],d.loc[['49ersRaiders']]['population'],(d.loc[['49ers','Raiders']]['W-L%'].sum())/2]
    d.loc[['49ersRaiders']] = SF
    d = d.dropna()
    a = d['population']
    b = d['W-L%']
    
    
    population_by_region = a # pass in metropolitan area population from cities
    win_loss_by_region = b # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    raise NotImplementedError()
nfl_correlation()

(0.004282119006786277, 0.9824115661872483)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
nfl_df=pd.read_csv("assets/nfl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
NFL = cities.iloc[:-1,[0,1,2]]
MLB = cities.iloc[:-1,[0,1,3]]
NHL = cities.iloc[:-1,[0,1,5]]
NBA = cities.iloc[:-1,[0,1,4]]
NFL = NFL.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population'})
MLB = MLB.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population'})
NBA = NBA.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population'})
NHL = NHL.rename(columns={'Metropolitan area':'local','Population (2016 est.)[8]':'population'})
w = re.compile(r'\[\w+\s\w+\]|\W')
NFL["NFL"] = NFL["NFL"].replace({w:''}, regex=True)
MLB["MLB"] = MLB["MLB"].replace({w:''}, regex=True)
NHL["NHL"] = NHL["NHL"].replace({w:''}, regex=True)
NBA["NBA"] = NBA["NBA"].replace({w:''}, regex=True)
NFL.drop(NFL[(NFL['NFL'] == "")].index, inplace=True)
MLB.drop(MLB[(MLB['MLB'] == "")].index, inplace=True)
NHL.drop(NHL[(NHL['NHL'] == "")].index, inplace=True)
NBA.drop(NBA[(NBA['NBA'] == "")].index, inplace=True)
mlb = mlb_df[(mlb_df['year']) == 2018]
mlb = mlb.iloc[:,[0,3]]
nfl = nfl_df[(nfl_df['year']) == 2018]
nfl = nfl.iloc[:,[12,13]]
nba = nba_df[(nba_df['year']) == 2018]
nba = nba.iloc[:,[0,3]]
nhl = nhl_df[(nhl_df['year']) == 2018]
#Limpieza NHL
nr = re.compile(r'\w+$|\w+\*$')
nhl["team"] = nhl["team"].replace({'\*':''}, regex=True)
nhl["team"] = nhl["team"].apply(lambda x: nr.findall(x)[0])
nhl["team"] = nhl["team"].replace({'Jackets':'BlueJackets','Leafs':'MapleLeafs',
                                   'Knights':'GoldenKnights','Ducks':'KingsDucks',
                                   'Wings':'RedWings'}, regex=True)
nhl = nhl.rename(columns={'team':'NHL'})
y = re.compile(r'^\w+')
NHL["NHL"] = NHL["NHL"].apply(lambda x: y.findall(x)[0])
WR = pd.pivot_table(nhl,index=["NHL"],values=['W', 'L' ],aggfunc=np.sum)
WR = WR.sort_values('NHL')
WR = WR.drop(['Division'], axis=0)
WR = WR.astype({'W': 'int32', 'L': 'int32'})
WR["WinRate"] = WR['W'] / (WR['W']+WR['L'])
nhl.set_index("NHL", drop=True, append=False, inplace=True, verify_integrity=False)
NHL.set_index("NHL",drop=True, append=False, inplace=True, verify_integrity=False)
f = pd.merge(WR, NHL, on='NHL', how='outer')
Vn = [f.loc[['Devils','Islanders','Rangers']]['L'].sum(),f.loc[['Devils','Islanders','Rangers']]['W'].sum(),
    f.loc[['Devils','Islanders','Rangers']]['WinRate'].sum(),f.loc[['RangersIslandersDevils']]['local'],
    f.loc[['RangersIslandersDevils']]['population']]
f.loc[['RangersIslandersDevils']]=Vn
Vla =[f.loc[['Kings','KingsDucks']]['L'].sum(),f.loc[['Kings','KingsDucks']]['W'].sum(),
    f.loc[['Kings','KingsDucks']]['WinRate'].sum(),f.loc[['KingsDucks']]['local'],
    f.loc[['KingsDucks']]['population']]
f.loc[['KingsDucks']]=Vla
f = f.dropna()
f = f.astype({'population':'int32'})
f = f.iloc[:,[2,3,4]]
f = f.reset_index()
f.set_index("local",drop=True, append=False, inplace=True, verify_integrity=False)
#Limpieza NBA
rnba = re.compile(r'\s\w+\s\(')
nba["team"] = nba["team"].replace({'\*':''}, regex=True)
nba["team"] = nba["team"].apply(lambda x: rnba.findall(x)[0])
nba["team"] = nba["team"].replace({'\(':''}, regex=True)
nba["team"] = nba["team"].replace({'\s':''}, regex=True)
nba['team'] = nba['team'].replace({'Blazers':'TrailBlazers','Clippers':'LakersClippers'},regex=True)
nba = nba.rename(columns={'team':'NBA'})
nba.set_index("NBA",drop=True, append=False, inplace=True, verify_integrity=False)
NBA.set_index("NBA",drop=True, append=False, inplace=True, verify_integrity=False)
nba = nba.sort_values('NBA')
NBA = NBA.sort_values('NBA')
nba = nba.astype({'W/L%': 'float32'})
NBA = NBA.astype({'population': 'int32'})
e = pd.merge(NBA, nba, on='NBA', how='outer')
LA = [e.loc['LakersClippers']['local']
     ,e.loc['LakersClippers']['population'],
     (e.loc[['LakersClippers','Lakers']]['W/L%'].sum())/2]
NY = [e.loc['KnicksNets']['local'],
    e.loc['KnicksNets']['population'],
    (e.loc[['Knicks','Nets']]['W/L%'].sum())/2]
e.loc['KnicksNets'] = NY
e.loc['LakersClippers'] = LA
e = e.dropna()
e = e.reset_index()
e.set_index("local",drop=True, append=False, inplace=True, verify_integrity=False)
#Limpieza MLB
rmlb = re.compile(r'\w+$')
mlb["team"] = mlb["team"].replace({'Boston Red Sox':'RedSox','Tampa Bay Rays':'Rays',
                                   'Toronto Blue Jays':'BlueJays','Chicago White Sox':'WhiteSox',}, regex=True)
mlb["team"] = mlb["team"].apply(lambda x: rmlb.findall(x)[0])
mlb = mlb.rename(columns={'team':'MLB'})
mlb.set_index("MLB", drop=True, append=False, inplace=True, verify_integrity=False)
MLB.set_index("MLB",drop=True, append=False, inplace=True, verify_integrity=False)
MLB = MLB.astype({'population':'int32'})
mlb = mlb.astype({'W-L%': 'float32'})
c = pd.merge(MLB, mlb, on='MLB', how='outer')
NY = [c.loc[['YankeesMets']]['local'],
      c.loc[['YankeesMets']]['population'],
      (c.loc[['Yankees','Mets']]['W-L%'].sum())/2]
c.loc[['YankeesMets']]=NY
LA = [c.loc[['DodgersAngels']]['local'],
      c.loc[['DodgersAngels']]['population'],
      (c.loc[['Dodgers','Angels']]['W-L%'].sum())/2]
c.loc[['DodgersAngels']]=LA
SF = [c.loc[['GiantsAthletics']]['local'],
      c.loc[['GiantsAthletics']]['population'],
      (c.loc[['Giants','Athletics']]['W-L%'].sum())/2]
c.loc[['GiantsAthletics']]=SF
C=[c.loc[['CubsWhiteSox']]['local'],
   c.loc[['CubsWhiteSox']]['population'],
   (c.loc[['Cubs','WhiteSox']]['W-L%'].sum())/2]
c.loc[['CubsWhiteSox']]=C
c = c.dropna()
c = c.reset_index()
c.set_index("local",drop=True, append=False, inplace=True, verify_integrity=False)
#limpieza NFL
rnfl = re.compile(r'\w+$|\w+\*|\w+\+')
y = re.compile(r'\w+\s\w+')
nfl["team"] = nfl["team"].apply(lambda x: rnfl.findall(x)[0])
nfl["team"] = nfl["team"].replace({w:''}, regex=True)
nfl["W-L%"] = nfl["W-L%"].replace({y:''}, regex=True)
a= nfl.drop(nfl[(nfl['W-L%'] == "")].index, inplace=True)
nfl["W-L%"] = nfl["W-L%"].replace({'\.':'0.'}, regex=True)
nfl = nfl.astype({'W-L%': 'float32'})
NFL = NFL.astype({'population':'int32'})
nfl = nfl.rename(columns={'team':'NFL'})
nfl.set_index("NFL", drop=True, append=False, inplace=True, verify_integrity=False)
NFL.set_index("NFL",drop=True, append=False, inplace=True, verify_integrity=False)
d = pd.merge(NFL, nfl, on='NFL', how='outer')
NY = [d.loc[['GiantsJets']]['local'],
      d.loc[['GiantsJets']]['population'],
      (d.loc[['Giants','Jets']]['W-L%'].sum())/2]
d.loc[['GiantsJets']]=NY
LA = [d.loc[['RamsChargers']]['local'],
      d.loc[['RamsChargers']]['population'],
      (d.loc[['Rams','Chargers']]['W-L%'].sum())/2]
d.loc[['RamsChargers']] = LA
SF = [d.loc[['49ersRaiders']]['local'],
      d.loc[['49ersRaiders']]['population'],
      (d.loc[['49ers','Raiders']]['W-L%'].sum())/2]
d.loc[['49ersRaiders']] = SF
d = d.dropna()
d = d.reset_index()
d.set_index("local",drop=True, append=False, inplace=True, verify_integrity=False)
fd = pd.merge(d, f, on='local', suffixes=('_1', '_2'))
print(f,d)
def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values